# File to experiment with different machine learning models.

In [1]:
from lib.DataObject import DataObject
import lib.DataObjectUtils as util
import torch
import pickle
import torch.nn as nn
from lib.DataHandler import DataAcquisitionHandler
from torch.utils.data import DataLoader, TensorDataset

pygame 2.5.1 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# load data from data/* using pickle
filename = 'data/handler_box_data_full_Oct_30_2023.pkl'
with open(filename, 'rb') as f:
    handler = pickle.load(f)

In [6]:
# Data filter visitor test

data = DataObject(handler.get_data())

data.accept(util.BandpassFilterVisitor(low=0.1, high=10))
data.accept(util.BandstopFilterVisitor(low=49, high=51))

key_data, box_data = data.get_data(decorator=util.MakeTensorWindowsDataDecorator())

sample = box_data[0]

print("List of samples:" ,len(box_data))
print("Sample - (channels, label):", len(box_data[0]))
print("Channels:", len(box_data[0][0]))
print("channel - [reading_1, ...]:", len(box_data[0][0][0]))
print("Sample example:", sample)

List of samples: 83
Sample - (channels, label): 2
Channels: 24
channel - [reading_1, ...] 248
Sample example: (tensor([[8.8000e+01, 8.9000e+01, 9.0000e+01,  ..., 7.7000e+01, 7.8000e+01,
         7.9000e+01],
        [4.4984e+03, 4.1819e+03, 4.4068e+03,  ..., 4.6585e+03, 4.6550e+03,
         4.1807e+03],
        [2.4223e+03, 2.4339e+03, 2.4448e+03,  ..., 2.3917e+03, 2.3875e+03,
         2.3846e+03],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.6987e+09, 1.6987e+09, 1.6987e+09,  ..., 1.6987e+09, 1.6987e+09,
         1.6987e+09],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]]), tensor(0))


In [7]:
# Dataset

class EEGDataset(torch.utils.data.Dataset):

    def __init__(self, data):
        self.data, self.labels = self.parse_data(data)
        self.window_size = self.data.shape[2]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index], self.labels[index]
    
    def parse_data(self, data):
        """
        Data comes in the form of a list of tuples (data, label)
        data is a 2D tensor (channels, readings)
        label is a 1D tensor (1)

        return:
        data: 3D tensor (samples, channels, readings)
        label: 1D tensor (samples)
        """

        data_list = []
        label_list = []

        for sample in data:
            data_list.append(sample[0])
            label_list.append(sample[1])

        return torch.stack(data_list), torch.stack(label_list)

In [8]:
# Data loader

dataset = EEGDataset(box_data)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

print(dataset.window_size)

RuntimeError: stack expects each tensor to be equal size, but got [24, 248] at entry 0 and [24, 249] at entry 10

In [1]:
# create p300Model
class EEGNet(torch.nn.Module):
    """
    Pytorch implementation of EEGNet

    Expecting input of shape (batch_size, channels, readings)
    batch_size: number of samples in a batch
    channels: number of channels in a sample (8)
    readings: number of readings in a channel (len())
    """

    def __init__(self):
        super(EEGNet, self).__init__()

        # block 1 nn.sequential
        self.block1 = torch.nn.Sequential(
            # Conv2D
            nn.Conv2d(1, 16, (1, 51), padding=0, bias=False),
            
            # Batch norm
            nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),

            # DepthwiseConv2D
            nn.Conv2d(16, 32, (2, 1), groups=16, bias=False),

            # Batch norm
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),

            # ELU Activation
            nn.ELU(alpha=1.0),

            # Avg Pooling 2D
            nn.AvgPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0),

            # Spacial Dropout 2D
            nn.Dropout2d(p=0.15)
        )

        # block 2 nn.sequential
        self.block2 = torch.nn.Sequential(
            # Separable Conv2D
            nn.Conv2d(32, 32, (1, 15), padding=0, bias=False),

            # Batch norm
            nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),

            # ELU Activation
            nn.ELU(alpha=1.0),

            # Avg Pooling 2D
            nn.AvgPool2d(kernel_size=(1, 8), stride=(1, 8), padding=0),

            # Spacial Dropout 2D
            nn.Dropout2d(p=0.15)
        )

        # Fully Connected Layer
        self.fc = nn.Linear(in_features=736, out_features=2, bias=True)

    def forward(self, x):
        # block 1
        x = self.block1(x)

        # block 2
        x = self.block2(x)

        # flatten
        x = x.view(x.shape[0], -1)

        # fc
        x = self.fc(x)

        return x

NameError: name 'torch' is not defined